# Tutorial 1 - Analyzing scRNA-seq data at the Protein Activity Level

<code style="background:lightgreen;color:black">Add description of the goal of this tutorial. Make sure that ways to load/install the package are updated. Also correct the path to the data (or include a dataset on GitHub, Zenodo or others). Also, fix the number of the sections.</code>

### Install Pyviper
Install `pyviper` from PyPI using pip. Alternatively, refer to the README in the current GitHub to install from the local directory.

In [ ]:
# !pip install pyviper

### Import modules

In [1]:
import pyther
import scanpy as sc
import anndata 
import pandas as pd

import warnings
warnings.filterwarnings("ignore", message=".*The 'nopython' keyword.*") # for jit decorator issue with sc.pp.neighbors (09/30/2023)

### Step 1. Load a gene expression "signature" at the single-cell level 
Load gene expression signature to be used as input to the `viper` function for Protein Activity inference. Store the gene expression signature into an [AnnData](https://anndata.readthedocs.io/en/latest/) object to enable interoperability with [scanpy](https://scanpy-tutorials.readthedocs.io/en/latest/#). Display matrix dimensions with cells on rows and features (genes) on columns (after transposition). The gene expression signature was generated from a population of malignant ductal cells from publicly avaiable data from [Peng et al., 2019](https://www.nature.com/articles/s41422-019-0195-y). Please, refer to Tutorial 2 for additional details on how to generate a gene expression signature.  

In [2]:
gene_expr_path = "/Users/lucazanella7/Desktop/ColumbiaProjects/pyther_test_data/subset_Peng/Ductal_2.csv"
gene_expr_signature = anndata.read_csv(gene_expr_path).T

gene_expr_signature

AnnData object with n_obs × n_vars = 11315 × 5000

### Step 2. Load a gene regulatory network inferred with ARACNe
Load and inspect lineage-specific gene regulatory network generated with the ARACNe3. See [ARACNe3](https://www.mdpi.com/1099-4300/25/3/542) and [ARACNe-AP](https://pubmed.ncbi.nlm.nih.gov/27153652/) for additional information on current ARACNe implementations. 

In [3]:
network_path = "/Users/lucazanella7/Desktop/ColumbiaProjects/pyther_test_data/subset_Peng/pruned_Ductal_2.tsv"
                            # path to tsv-formatted ARACNe network
    
network = pd.read_csv(network_path, delimiter="\t")

Display the first 5 interactions in the regulatory network. `mor` and `likelihood` represent the mode of regulation of the given regulator-target pair and the likelihood of the interaction, respectively.

In [4]:
network.head()

,regulator,target,mor,likelihood
0,AATF,CDC42SE2,0.005831,0.999934
1,AATF,EIF4EBP2,0.019100,0.999868
2,AATF,DDB1,0.019182,0.999802
3,AATF,COPB2,0.064825,0.999736
4,AATF,MDM4,0.023039,0.999670


Other algorithms can be used to generate gene regulatory networks, but we recommend postprocessing the output to the suitable dataframe format, with columns displayed above.

Convert the pandas DataFrame to an object of class `Interactome` to enable easier manipulation. Type `help(pyther.Interactome)` to see available methods.

In [5]:
network_interactome = pyther.Interactome('malignant_ductal_interactome', network)

Filter out targets in the `Interactome` that are not present in the gene expression matrix. This step is recommended before running `viper`. 

In [6]:
network_interactome.filter_targets(gene_expr_signature.var_names)

As an example, display the number of targets of a couple of regulators, MYC and SERPINA12.

In [7]:
n_MYC = len(network_interactome.get_reg('MYC')) # number of MYC targets in the network
n_SERPINA12 = len(network_interactome.get_reg('SERPINA12')) # number of SERPINA12 targets in the network

print("The number of targets of MYC and SERPINA12 is " + str(n_MYC) + " and " + str(n_SERPINA12) + ", respectively.")

The number of targets of MYC and SERPINA12 is 118 and 144, respectively.


### Step 3. Convert the gene expression signature into a protein activity matrix using viper
`viper` transforms a gene expression signature into a protein activity matrix by performing enrichment analysis on regulons. Two methods for enrichment analysis are allowed: [aREA](https://www.nature.com/articles/ng.3593) (default) and [NaRnEA](https://www.mdpi.com/1099-4300/25/3/542) (by setting `enrichment="narnea"`).

#### Method 1 - protein activity inference using aREA
As we have seen (section above), different regulators can potentially have different number of targets. We prune each regulon to have the same number of targets (50 in this case). This step is advisable to avoid regulators with an exceedingly number of targets to dominate those with fewer when computing the NES, thus making NES scores comparable.

In [8]:
network_pruned = network_interactome.copy() # generate a copy of the unpruned network before pruning it
network_pruned.prune(max_targets=50,eliminate=True) # prune interactome to have exactly 50 targets

Now all the regulators in the network have exactly 50 transcriptional targets.

In [9]:
n_MYC = len(network_pruned.get_reg('MYC')) # number of MYC targets in the network
n_SERPINA12 = len(network_pruned.get_reg('SERPINA12')) # number of SERPINA12 targets in the network

print("Number of MYC targets: " + str(n_MYC) + "\nNumber of SERPINA12 targets: " + str(n_SERPINA12))

Number of MYC targets: 50
Number of SERPINA12 targets: 50


<div class="alert alert-block alert-success">
<b></b> Run `viper` to compute the protein activity matrix (aREA method).
</div>
 
The mandatory inputs to `viper` are a gene expression signature and a gene regulatory network. We will set the output output to be an ndarray (The default output would be an AnnData object).

In [10]:
ProtAct_aREA = pyther.viper(gex_data=gene_expr_signature, # gene expression signature
                             interactome=network_pruned, # gene regulatory network
                             enrichment = "area",
                             output_as_anndata=False,
                             njobs=1,
                             verbose=False)

`ProtAct_aREA` contains the activity of each regulatory protein computed as a Normalized Enrichment Score (NES) for each single cell.

In [11]:
ProtAct_aREA # display the protein activity matrix 

regulator,AATF,ABCA1,ABCA12,ABCA3,ABCA7,ABCA8,ABCB1,ABCB11,ABCB4,ABCB5,...,ZSWIM5,ZSWIM6,ZSWIM7,ZWINT,ZXDA,ZXDB,ZXDC,ZYX,ZZEF1,ZZZ3
T1_AACACGTCAATGACCT,0.003478,0.021827,-0.000146,-1.892847,0.012870,-1.626956,0.020836,0.295218,-0.043554,0.051283,...,-0.981025,-0.301464,-0.477378,1.952636,0.020272,-0.017226,-0.675347,0.005074,-0.036858,1.615052
T1_AACCATGCACAACTGT,0.021164,0.022861,0.017899,-1.915731,0.143748,-1.285993,-0.007112,-0.424700,1.630497,0.039928,...,-0.384865,0.751079,-0.384921,0.632195,0.013861,0.035514,0.486563,0.026284,-0.377175,1.666068
T1_AACCATGTCTGATTCT,0.005886,0.010786,0.003141,-1.151431,-0.035641,-1.060720,0.128850,0.623142,-0.773390,-0.041436,...,-0.281773,0.046981,0.844734,-0.710689,-0.006223,0.176814,1.029464,-0.006727,-0.538143,0.006137
T1_AACCATGTCTGTCCGT,0.014610,0.037640,0.041812,1.548044,0.025776,1.055263,-0.100038,0.415668,-0.573411,-0.032747,...,-0.602362,-0.105102,0.964380,0.178080,-0.018838,0.020599,0.013466,0.084279,-0.448729,0.469023
T1_AACCGCGGTACTCGCG,0.017482,0.028096,0.002755,-2.027166,-0.017121,-0.850260,-0.018189,0.404971,-0.036865,-0.076835,...,-0.304488,0.410485,1.484359,-0.693966,-0.004976,0.020983,-0.185549,0.013146,0.002211,-0.040599
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
T24_TTTCCTCCAGACAAGC,0.017641,0.005050,0.042925,0.767404,-0.031865,-1.046213,0.016363,1.642829,-0.893687,0.019125,...,0.751845,-0.267433,-1.031529,-0.688393,0.025190,-0.034690,-1.727441,-0.059350,-0.011701,0.060889
T24_TTTGCGCCAATCGAAA,-0.011175,0.010350,-0.011520,1.265818,-0.006328,-1.184348,0.039789,-1.155374,-0.138744,-0.218545,...,-0.803185,0.014058,0.769134,-0.971193,0.002922,-0.011592,-0.635144,-0.032277,-0.012269,0.112088
T24_TTTGCGCTCTTATCTG,0.021221,0.016244,-0.003535,2.289876,0.036617,-1.421605,0.981048,0.009560,-0.604974,-0.092936,...,-0.331045,-0.111880,0.369194,-0.733554,0.024871,0.013581,-0.169933,0.013973,-0.019886,0.001815
T24_TTTGGTTAGACACTAA,0.063437,0.018660,0.004487,1.473815,-0.000105,-1.784739,0.107552,0.216333,0.025507,0.027550,...,0.097063,0.190448,0.715559,-0.639196,0.038621,0.026653,-0.510227,0.062492,0.170191,1.378558


#### Method 2 - protein activity inference using (matrix)-NaRnEA

<div class="alert alert-block alert-success">
<b></b> Run `viper` to compute the protein activity matrix (matrix-NaRnEA method).
</div>
 

Compute the protein activity using `viper` with NaRnEA as the enrichment method. Unlike aREA, NaRnEA can run with regulons of different sizes, i.e. different number of targets per regulator. Store the output in an `AnnData` object (default).

In [12]:
ProtAct_NaRnEA = pyther.viper(gex_data=gene_expr_signature, # gene expression signature
                                 interactome=network_interactome, # gene regulatory network
                                 enrichment = "narnea",
                                 njobs=1,
                                 verbose=False)

Show the output type and length of `ProtAct_NaRnEA`.

In [13]:
ProtAct_NaRnEA # display the protein activity matrix as AnnData object 

AnnData object with n_obs × n_vars = 11315 × 6593
    uns: 'gex_data'
    layers: 'pes'

When using NaRnEA, `viper` returns an additional layer, `pes` that stores the Proportional Enrichment Scores (PES) for regulator, a measure bound in the interval $-1 \le PES \le 1$ that can be used as a measure of effect size. For further details, see [Griffin et al., 2022](https://pubmed.ncbi.nlm.nih.gov/36981431/).

Display NES protein activity matrix.


In [15]:
ProtAct_NaRnEA.to_df() # NES matrix

,AAMP,AATF,ABCA1,ABCA12,ABCA3,ABCA7,ABCA8,ABCB1,ABCB11,ABCB4,...,ZSWIM5,ZSWIM6,ZSWIM7,ZWINT,ZXDA,ZXDB,ZXDC,ZYX,ZZEF1,ZZZ3
T1_AACACGTCAATGACCT,5.951375,-6.182347,-2.507142,-2.253130,-2.496547,7.366656,-2.128077,-1.860151,1.616963,-2.362606,...,1.166578,4.976091,-5.864811,-6.763819,1.716382,-3.630205,-3.348021,7.975582,3.442741,-5.313166
T1_AACCATGCACAACTGT,-7.905461,-7.662926,2.121460,-1.175113,-2.468094,-7.462323,-2.433052,1.263038,-1.058242,-2.084751,...,1.246235,-5.531142,-6.544208,-6.002681,-1.430644,-3.714284,2.529195,-6.750428,-3.783460,-6.636886
T1_AACCATGTCTGATTCT,-4.467884,-5.220842,-3.186879,-1.261798,1.777065,-6.956938,-2.696425,-1.186100,1.051735,2.481672,...,2.060335,6.235016,5.617225,-6.921654,2.338369,-3.725552,1.978702,6.755377,4.470265,-4.883226
T1_AACCATGTCTGTCCGT,6.219553,-6.974754,-2.578445,-1.740226,-2.163299,-9.057311,2.081327,1.035042,1.236982,2.147826,...,1.243236,5.120960,-6.440320,-7.196440,-0.828349,5.631126,-2.386871,7.764003,-3.036024,-5.383112
T1_AACCGCGGTACTCGCG,5.906399,-6.775007,-3.700379,1.809953,2.655762,-6.888924,-1.019356,1.441171,-1.354680,2.349706,...,1.242376,6.220801,7.076439,-6.452016,1.313020,3.304231,1.939388,6.313558,-3.967383,5.199342
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
T24_TTTCCTCCAGACAAGC,-3.179003,3.849975,-3.416965,1.749656,-1.612980,5.309050,2.132325,-0.849799,-2.520791,2.597153,...,-2.117735,-4.934680,-3.930921,11.898617,-1.004773,2.930437,-2.374563,2.873573,4.134772,3.602358
T24_TTTGCGCCAATCGAAA,4.136378,6.295573,-2.594259,-2.241861,1.256665,6.880444,-1.721376,1.454858,-1.196661,-2.111783,...,-2.982656,6.261072,-5.760741,-5.334735,-1.573474,-4.339206,-3.580222,-5.140140,4.642709,-4.783969
T24_TTTGCGCTCTTATCTG,7.836391,-7.106687,-2.479225,2.209056,-1.289580,-7.024376,-2.280278,-1.358428,1.190137,2.781669,...,-1.963767,-6.282956,5.468326,-6.062498,-1.796486,3.518511,-1.771200,5.989233,-5.568146,-5.535362
T24_TTTGGTTAGACACTAA,6.241811,6.393485,-2.661585,1.277664,1.850238,-8.116560,2.631627,2.454699,-1.170700,-2.092703,...,1.939028,6.934541,-6.359559,-5.469336,2.603914,4.882271,-2.601518,5.996581,-3.996712,5.538172


Display the PES protein activity matrix.

In [16]:
ProtAct_NaRnEA.to_df(layer="pes") # PES matrix

,AAMP,AATF,ABCA1,ABCA12,ABCA3,ABCA7,ABCA8,ABCB1,ABCB11,ABCB4,...,ZSWIM5,ZSWIM6,ZSWIM7,ZWINT,ZXDA,ZXDB,ZXDC,ZYX,ZZEF1,ZZZ3
T1_AACACGTCAATGACCT,0.306357,-0.408439,-0.110749,-0.098052,-0.097408,0.265594,-0.084639,-0.073236,0.059162,-0.088962,...,0.043409,0.198336,-0.375905,-0.241983,0.065417,-0.157345,-0.146673,0.350361,0.119426,-0.338686
T1_AACCATGCACAACTGT,-0.583017,-0.525114,0.074893,-0.049983,-0.094094,-0.389278,-0.094424,0.045944,-0.037941,-0.075374,...,0.044596,-0.311580,-0.436319,-0.222803,-0.053790,-0.162292,0.093505,-0.455874,-0.164573,-0.439685
T1_AACCATGTCTGATTCT,-0.303750,-0.328643,-0.140421,-0.055874,0.068011,-0.333324,-0.109304,-0.047926,0.039943,0.097610,...,0.079386,0.276174,0.285318,-0.235747,0.092640,-0.158800,0.082493,0.335159,0.168653,-0.295898
T1_AACCATGTCTGTCCGT,0.299271,-0.471106,-0.113513,-0.074641,-0.083175,-0.465730,0.075041,0.038304,0.044103,0.080168,...,0.045148,0.190969,-0.422768,-0.263264,-0.031542,0.187879,-0.103621,0.315866,-0.131508,-0.351251
T1_AACCGCGGTACTCGCG,0.308358,-0.445873,-0.163710,0.074094,0.097406,-0.345215,-0.040706,0.055225,-0.050565,0.090121,...,0.046497,0.251425,0.325877,-0.229937,0.050358,0.117484,0.076919,0.281765,-0.170872,0.239255
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
T24_TTTCCTCCAGACAAGC,-0.211809,0.213254,-0.150576,0.076473,-0.064643,0.224186,0.085493,-0.034781,-0.097921,0.103678,...,-0.083247,-0.252918,-0.233777,0.354508,-0.040819,0.116120,-0.105338,0.150401,0.162042,0.191371
T24_TTTGCGCCAATCGAAA,0.239280,0.335720,-0.114056,-0.099273,0.048265,0.279497,-0.069788,0.057882,-0.045866,-0.082638,...,-0.115675,0.279957,-0.347729,-0.180363,-0.063095,-0.184255,-0.157913,-0.308280,0.176416,-0.287722
T24_TTTGCGCTCTTATCTG,0.410541,-0.466455,-0.109568,0.090523,-0.050495,-0.351040,-0.091036,-0.053741,0.043833,0.106714,...,-0.074136,-0.340810,0.252764,-0.215466,-0.070249,0.125399,-0.077752,0.268401,-0.239418,-0.350437
T24_TTTGGTTAGACACTAA,0.313877,0.296471,-0.117585,0.051357,0.066622,-0.412124,0.097025,0.092608,-0.043178,-0.078175,...,0.071605,0.270096,-0.411751,-0.197534,0.098410,0.168612,-0.113742,0.256464,-0.172762,0.245280


Notice that when setting the `viper` output to be an object of class `AnnData`, the input gene expression `AnnData` is stored under the attribute `gex_data`:

In [17]:
ProtAct_NaRnEA.uns["gex_data"][0:5,0:5].to_df()

,RP11-206L10.2,RP11-54O7.3,HES4,ISG15,AGRN
T1_AACACGTCAATGACCT,-0.068999,-0.023360,-0.616169,-1.668614,7.155818
T1_AACCATGCACAACTGT,-0.051544,-0.014172,2.159164,-0.277659,-0.480383
T1_AACCATGTCTGATTCT,-0.081841,-0.030217,-0.731943,-0.967582,0.156287
T1_AACCATGTCTGTCCGT,-0.053397,-0.015141,-0.466018,0.654629,-0.501795
T1_AACCGCGGTACTCGCG,-0.073019,-0.025496,-0.653355,-1.721549,1.645273


### Step 4. Analyze single-cells at the Protein Activity level 
We present some basic, but instructive analyses at the protein activity matrix. Start by running Principal Components Analysis (PCA) on the PES matrix, to reduce the dimensionality of the dataset. Use the PCA function avaiable through the `tl` module that provides wrappers to `scanpy.tl` which simplifies the transformation of protein activity data. For instance, functions from `tl` allow subsetting the class of regulatory proteins to analyze. 

Run PCA using by projecting the PES of transcriptional regulators (TFs) and co-transcriptional regulators (coTFs) by setting `filter_by_feature_groups=["tfs", "cotfs"]`. Possible options for this parameter are `["tfs", "cotfs", "sig", "surface"]`, where the latter refer to signaling proteins and surface markers, respectively.

In [34]:
pyther.tl.pca(ProtAct_NaRnEA, layer="pes", filter_by_feature_groups=["tfs", "cotfs"], zero_center=True,  svd_solver='arpack')

FileNotFoundError: [Errno 2] No such file or directory: '/Users/lucazanella7/Desktop/ColumbiaProjects/pyther/pyther/data/regulatorIDs/tfs-hugo.txt'

Compute the neighbors graph of cells using the PCA representation of the protein activity matrix. For sake of simplicity, we set 10 nearest neightbors and 50 principal components. Typically, these parameters need tuning.

In [ ]:
sc.pp.neighbors(ProtAct_NaRnEA, metric="euclidean", n_neighbors=10, n_pcs=50)

Cluster at protein activity using the Leiden algorithm with `resolution=0.2` (in real applications this parameter should be optimized) <code style="background:lightgreen;color:black">How does the leiden option with tfs and cotfs exactly work?.</code>

In [ ]:
sc.tl.leiden(ProtAct_NaRnEA, resolution=0.2, n_iterations=-1)

Generate a [UMAP](https://arxiv.org/abs/1802.03426) embedding of the previously computed graph, filtering by `"tfs"` and `"cotfs"`. In this example, we focus on TFs and coTFs because we are interested in proteins involved in transcriptional regulation and that are mechanistic determinants of cell state, as shown in [Paull et al., 2021](https://www.sciencedirect.com/science/article/pii/S0092867420316172?via%3Dihub) and [Califano & Alvarez, 2016](https://www.nature.com/articles/nrc.2016.124) .

In [ ]:
pyther.tl.umap(ProtAct_NaRnEA, filter_by_feature_groups=['tfs', 'cotfs'])

Display the 2-dimensional UMAP embedding with the corresponding function from the `pyther.pl`, a module that provides wrappers for `scanpy.pl` functions for enhanced visualization of key protein activity features. Color by cluster.  

In [ ]:
sc.set_figure_params(dpi=90)
n_cells = ProtAct_NaRnEA.shape[0]
dot_size = 120000 / n_cells*2
pyther.pl.umap(ProtAct_NaRnEA, color='leiden', palette="Set1", size=dot_size,add_outline=True)

Find the regulatory proteins predicted to be most active for each cell type by integrating the NES of each regulatory protein across all cells of a given cell type using the Stouffer's method: $$ NES^C_j = \frac{\sum_{k \in C}NES_{k,j}}{\sqrt{n^C}} $$ where $NES^C_j$ denotes the integrated NES for regulator $j$ in cluster/cell type $C$, $NES_{k,j}$ is the NES of regulator $j$ in cluster/cell type $C$ and $n^C$ is the number of cells in cluster $C$.

In [ ]:
NES_integrated = pyther.compute_cluster_stouffer_anndata(ProtAct_NaRnEA, "leiden") # Stouffer integration 

Analogously, `compute_cluster_stouffer`, can be used to perform the same operations in Pandas DataFrames. Remove ribosomal proteins from the above matrix (ribosomal protein genes are among the most highly expressed genes in most cell types).

In [ ]:
filtered_proteins = NES_integrated.var_names[~NES_integrated.var_names.str.startswith('RP')].to_list() # Exclude columns starting with 'RP' from integrated matrix
NES_integrated = NES_integrated[:,filtered_proteins] # remove ribosomal proteins from the AnnData object

Slice `NES_integrated` to contain only on TFs and coTFs and focus only on these proteins. 

In [ ]:
TFs_list = pyther.load.load_TFs(species="human") # load list of TFs
coTFs_list = pyther.load.load_coTFs(species="human") # load list of coTFs
TFs_coTFs = TFs_list + coTFs_list # concatenate TFs and coTFs

NES_integrated = NES_integrated[:,NES_integrated.var_names.isin(TFs_coTFs)] # slice NES_integrated to have only TFs and coTFs

Extract the 5 most activated regulatory protein in each cluster

In [ ]:
active_proteins = NES_integrated.to_df().apply(lambda row: row.nlargest(10).index.tolist(), axis=1)  # the top 10 most activated proteins in each cell type
active_proteins

Show the top activated proteins on a heatmap

In [ ]:
protein_set = active_proteins.sum() # protein set to display on the heatmap
ax = pyther.pl.heatmap(ProtAct_NaRnEA, var_names=protein_set, groupby="leiden",vmin=-15,vmax=15, vcenter=0,cmap="RdBu_r",dendrogram=False,swap_axes=True, show_gene_labels=False)

In [ ]:
pyther.pl.dotplot(ProtAct_NaRnEA, var_names=protein_set, layer="pes",groupby='leiden', cmap="RdBu_r", vcenter=0,dendrogram=False)